<a href="https://colab.research.google.com/github/mr7495/COVID-CT-Code/blob/master/Automated_covid_detector_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi #show the allocated GPU

In [0]:
#Connect your Google Drive 
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#upgrade and install essential libraries
#Install essential libraries
!pip install zipfile36
!pip install git+https://github.com/mr7495/RetinaNet

In [0]:
import keras
import numpy as np
import cv2
import os
import random
import shutil
import pandas as pd
import csv
import zipfile
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import RandomNormal
import keras.backend as k
from sklearn.utils import shuffle
import io
from PIL import Image as pil_image
from keras_retinanet import layers
import keras.backend as k
import keras_retinanet

Using TensorFlow backend.


In [0]:
# Add the COVID-CTset to your drive through this link:
#https://drive.google.com/drive/folders/1xdk-mCkxCDNwsMAk2SGv203rY1mrbnPB?usp=sharing

In [0]:
archive = zipfile.ZipFile('drive/My Drive/COVID-CTset/COVID-CTset.zip') #Path to the shared dataset
for file in archive.namelist():
     archive.extract(file, './data') #Extract the data

In [0]:
fold_num=1 #Select Fold Number (You can predict and evaluate the patients in every folds)
shape=(512,512,1) #shape of the dataset images (in TIFF format)

In [0]:
#Create the reader
test_df = pd.read_csv('drive/My Drive/COVID-CTset/CSV/test{}.csv'.format(fold_num))#raed test csv file (For evaluating the final version of the trained network)

In [0]:
#The fully automated patient classification validation
Path='Path to the trained models' # You can use this trained model : 'drive/My Drive/COVID-CTset/trained_networks/FPN-fold1.hdf5'

full_reports=[]
k.clear_session() #clear keras backend
custom_object={'UpsampleLike': keras_retinanet.layers._misc.UpsampleLike} #Upsampling layer that was used in feature pyramid network
model_name=Path
net=keras.models.load_model(Path, custom_objects=custom_object)


patients={'normal':{},'covid':{}} #Dictionary for refering to each patient images path
for r,d,f in os.walk('data'): 
    for file in f:
        if'.tif' in file:
            full_add=os.path.join(r,file)
            if 'SR_2' in full_add: #Select only SR_2 folder
              index1=img_name.index('patient')
              index2=img_name.index('_SR')
              if 'covid' in full_add:
                if full_add[index1:index2-1] not in patients['covid']: #full_add[index1:index2-1] shows the patient id
                    patients['covid'][full_add[index1:index2-1]]=[]
                patients['covid'][full_add[index1:index2-1]].append(full_add) #add the images of a patient
              elif 'normal' in full_add:
                if full_add[index1:index2-1] not in patients['normal']:
                    patients['normal'][full_add[index1:index2-1]]=[]
                patients['normal'][full_add[index1:index2-1]].append(full_add)


tp=0 #All True Positives
fp=0 #All False Positives
ctp=0 #COVID True Postivies
cfp=0 #COVID False Postivies
ntp=0 #Normal True Postivies
nfp=0 #Normal False Postivies
covid_label=0 # index of COViD-19 class 
normal_label=1  # index of normal class 
results={}
for patientid in patients['normal']:
  results[patientid]={'covid-detect':0,'normal-detect':0,'gt':'normal','decision':0}
  for img_name in patients['normal'][patientid]: #read images of a normal person CT scans
    img=cv2.imread(img_name,cv2.IMREAD_UNCHANGED)
    pred_ind=np.argmax(net.predict(np.expand_dims(np.expand_dims(img,axis=0),axis=3))[0]) #predicted label
    if pred_ind==normal_label:
      results[patientid]['normal-detect']+=1
    else:
      results[patientid]['covid-detect']+=1
  all_imgs_num=len(patients['normal'][patientid]) 
  normal_detect_num=results[patientid]['normal-detect']
  covid_detect_num=results[patientid]['covid-detect']
  if covid_detect_num>=0.1*all_imgs_num: #if at least 10% of a patient CT scans be classified as covid
    results[patientid]['decision']='covid'
    fp+=1 
    cfp+=1
  else:
    results[patientid]['decision']='normal'
    tp+=1
    ntp+=1
###########################################
for patientid in patients['covid']: #read images of a normal person CT scans
  results[patientid]={'covid-detect':0,'normal-detect':0,'gt':'covid','decision':0}
  for img_name in patients['covid'][patientid]:
    img=cv2.imread(img_name,cv2.IMREAD_UNCHANGED)
    pred_ind=np.argmax(net.predict(np.expand_dims(np.expand_dims(img,axis=0),axis=3))[0]) #predicted label
    if pred_ind==covid_label:
      results[patientid]['covid-detect']+=1
    else:
      results[patientid]['normal-detect']+=1
  all_imgs_num=len(patients['covid'][patientid])
  normal_detect_num=results[patientid]['normal-detect']
  covid_detect_num=results[patientid]['covid-detect']
  if covid_detect_num>=0.1*all_imgs_num: #if at least 10% of a patient CT scans be classified as covid
    results[patientid]['decision']='covid'
    tp+=1
    ctp+=1
  else:
    results[patientid]['decision']='normal'
    fp+=1
    nfp+=1
full_reports.append([model_name,tp,fp,ctp,cfp,ntp,nfp])
print('tp: ',tp,'fp: ',fp) 
with open('{}-results.csv'.format(model_name), mode='w',newline='') as csv_file: #write the results
    csvwriter = csv.writer(csv_file, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['model_name','tp','fp','ctp','cfp','ntp','nfp'])
    for row in full_reports:
        csvwriter.writerow(row)  

In [0]:
# Use this code to see which patient has been detected wrong
for key in results:
  if results[key]['gt']!=results[key]['decision']:
    print(key)
    print(results[key])


"\nfor key in results:\n  if results[key]['gt']!=results[key]['decision']:\n    print(key)\n    print(results[key])\n"